In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from collections import Counter
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import time

In [ ]:
sklearn.__version__

In [ ]:
map_df = pd.read_csv('../QIIME2/mapping_file/mapping_file.txt', index_col=0, sep = '\t').sort_index()
map_df[:5]

In [ ]:
# Download L7s from taxa-bar-plots.qzv
# Delete env columns for each L7 file
alpha_df = pd.read_csv('../QIIME2/exported-table-taxonomy/table.from_biom_w_taxonomy.txt',  skiprows =1, index_col=0, sep ='\t')
rf_df = alpha_df.T
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
map_df_numeric = map_df.select_dtypes(include=numerics)
# rf_df = pd.merge(rf_df, map_df_numeric, left_index=True, right_index=True)
rf_df.head()

In [ ]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), stats.sem(a)
    h = se * stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [ ]:
# anova_df = pd.DataFrame()
# mean_ci_df = pd.DataFrame()
# statistics = []
# p_values = []
# means ={'PG':[],'M': []}
# lowers = {'PG':[],'M': []}
# uppers = {'PG':[],'M': []}

# for column in rf_df.columns.values:
    
#     pg = rf_df[rf_df.index.isin(map_df[map_df['Landform'] == 'PG'].index)][column]
#     m = rf_df[rf_df.index.isin(map_df[map_df['Landform'] == 'M'].index)][column]
    
#     statistic, p_value = stats.f_oneway(pg,m)
#     statistics.append(statistic)
#     p_values.append(p_value)
#     for i in ['PG','M']:
#         mean_confidence = mean_confidence_interval(rf_df[map_df['Landform'] == i][column])
#         means[i].append(mean_confidence[0])
#         lowers[i].append(mean_confidence[1])
#         uppers[i].append(mean_confidence[2])

# anova_df['Columns'] = rf_df.columns.values
# anova_df['Statistic'] = statistics
# anova_df['P_value'] = p_values
# anova_df['PG_mean'] = means['PG']
# anova_df['PG_lower'] = lowers['PG']
# anova_df['PG_upper'] = uppers['PG']
# anova_df['M_mean'] = means['M']
# anova_df['M_lower'] = lowers['M']
# anova_df['M_upper'] = uppers['M']
# anova_df.sort_values('Statistic', ascending=False, inplace=True)

In [ ]:
# anova_df.head()

In [ ]:
# anova_df.loc[392][0]

In [ ]:
# list(anova_df['Columns'])

In [ ]:
# moore_lm = ols('D_0__Bacteria;D_1__Chloroflexi;D_2__Ktedonobacteria;D_3__Ktedonobacterales;D_4__Ktedonobacteraceae;D_5__JG30a-KF-32;__ ~ Landform',data=rf_df).fit()
# table = sm.stats.anova_lm(moore_lm, typ=2) # Type 2 Anova DataFrame
# print(table)

In [ ]:
print(rf_df.shape)
print(map_df.shape)

In [ ]:
# start = time.time()
# accuracy_list = []
# random_chance_list = []
# for i in range(50):
#     X_train, X_test, Y_train, Y_test = train_test_split(rf_df[rf_df.index.isin(map_df.index)].values,
#                                                         map_df[map_df.index.isin(rf_df.index)]['genotype_1'].values, test_size=0.3, random_state=i)
#     param_grid = { 
#         'n_estimators': [200, 500],
#         'max_features': ['auto', 'sqrt', 'log2'],
#         'max_depth' : [2,3,4,5,6,7,8,9,10],
#         'criterion' :['gini', 'entropy']
#     }
#     rfc=RandomForestClassifier(random_state=42)
#     CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
#     CV_rfc.fit(X_train, Y_train)
#     predicted = CV_rfc.predict(X_test)
# end = time.time()
# print(end - start)

In [ ]:
print(CV_rfc.best_params_)

In [10]:
start = time.time()
accuracy_list = []
random_chance_list = []
predicted_category = 'genotype_1'
for i in range(50):
    X_train, X_test, Y_train, Y_test = train_test_split(rf_df[rf_df.index.isin(map_df.index)].values,
                                                        map_df[map_df.index.isin(rf_df.index)][predicted_category].values, test_size=0.3, random_state=i)
    clf = RandomForestClassifier(n_estimators=200, max_features='log2', max_depth=9, random_state=42, criterion= 'gini')
    predicted = clf.predict(X_test)
    accuracy = accuracy_score(Y_test, predicted)
#     if predicted_category == 'Landform':
# #     print 'Accuracy:\t' + str(accuracy)
#         sum_counter = float(Counter(Y_test)['PG'] + Counter(Y_test)['M'])
#         random_chance = round((Counter(Y_test)['PG']/sum_counter)**2+(Counter(Y_test)['M']/(sum_counter))**2,2)
#     elif predicted_category == 'Depth_PG_M':
#         sum_counter = float(Counter(Y_test)['PGA'] + Counter(Y_test)['MA'] + Counter(Y_test)['PGB'] + Counter(Y_test)['MB'])

#         random_chance = round((Counter(Y_test)['PGA']/sum_counter)**2+(Counter(Y_test)['MA']/(sum_counter))**2 +
#                               (Counter(Y_test)['PGB']/sum_counter)**2+(Counter(Y_test)['MB']/sum_counter)**2,2)
# #     print 'Random:\t\t' + str(random_chance)
#     random_chance_list.append(random_chance)
    accuracy_list.append(accuracy)
end = time.time()
print(end - start)

9.521011590957642


In [ ]:
# Results of genotype
# n_estimators=200, max_features='log2', max_depth=9, random_state=42, criterion= 'gini'
# 33% accuracy with genotype
# Similar importance features as those identified by LDA
# Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;__ Importance: 0.05
# Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__ORVIN GLO3D Importance: 0.05
# Variable: k__fungi;p__Glomeromycota;__;__;__;__;__ Importance: 0.05
# Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__MO-G17 Importance: 0.04
# Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__Yamato09 A2 Importance: 0.04
# Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__Glo45 Importance: 0.04
# Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__MO-G8 Importance: 0.04

In [16]:
# rf = {'Random':random_chance_list, 'Accuracy':accuracy_list}
# df_rf_anova = pd.DataFrame(data = rf)
# statistic, p_value = stats.f_oneway(df_rf_anova['Random'],df_rf_anova['Accuracy'])
# print(statistic)
# print(p_value)

1154.0883734226334
5.111079500256419e-56


In [11]:
# Just two
print(sum(accuracy_list)/len(accuracy_list)*100)
# print(sum(random_chance_list)/len(random_chance_list)*100)

33.023255813953504


In [18]:
# # All 4 categories
# print(sum(accuracy_list)/len(accuracy_list)*100)
# print(sum(random_chance_list)/len(random_chance_list)*100)
# # 69.4
# # 30.599999999999998

In [12]:
len(accuracy_list)

50

In [14]:
# map_df[map_df.index.isin(rf_df.index)]['Depth_PG_M'].value_counts()

In [15]:
print(metrics.classification_report(Y_test, predicted))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         1
           3       0.17      0.50      0.25         2
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         3
           6       1.00      0.14      0.25         7
           7       0.00      0.00      0.00         4
           8       0.20      0.17      0.18         6
           9       0.00      0.00      0.00         3
          10       0.60      0.75      0.67         4
          11       0.33      0.20      0.25         5
          12       0.17      0.50      0.25         2

    accuracy                           0.19        43
   macro avg       0.21      0.19      0.15        43
weighted avg       0.30      0.19      0.18        43



/home/bostevens/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
len(Y_train)

99

In [17]:
# estimator = clf.estimators_[9]
# from sklearn.tree import export_graphviz
# # Export as dot file
# export_graphviz(estimator, 
#                 out_file='tree.dot', 
#                 feature_names = list(rf_df.columns),
#                 class_names = Y_train,
#                 rounded = True, proportion = False, 
#                 filled = True)
# from subprocess import call
# call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])
# from IPython.display import Image
# Image(filename = 'tree.png')

In [18]:
# Use numpy to convert to arrays
import numpy as np
# Labels are the values we want to predict
labels = map_df[map_df.index.isin(rf_df.index)][predicted_category].values
# Remove the labels from the features
# axis 1 refers to the columns
features= rf_df
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)
# Get numerical feature importances
importances = list(clf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];


Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;__ Importance: 0.05
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__ORVIN GLO3D Importance: 0.05
Variable: k__fungi;p__Glomeromycota;__;__;__;__;__ Importance: 0.05
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__MO-G17 Importance: 0.04
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__Yamato09 A2 Importance: 0.04
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__Glo45 Importance: 0.04
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae;g__Glomus;s__MO-G8 Importance: 0.04
Variable: k__fungi;p__Glomeromycota;c__Paraglomeromycetes;o__Paraglomerales;f__Paraglomeraceae;g__Paraglomus;__ Importance: 0.04
Variable: k__fungi;p__Glomeromycota;c__Glomeromycetes;o__Glomerales;f__Glomeraceae

In [25]:
feature_importances_asav_count = pd.merge(pd.DataFrame(feature_importances).set_index(0), pd.DataFrame(rf_df.sum(axis = 0)), left_index=True, right_index=True)
